In [1]:
import numpy as np
import pandas as pd
import xlsxwriter

import gurobipy as gp
from gurobipy import GRB


np.random.seed(0)

In [2]:
####################### INDEX ################################

j = 3  #Customer sub index
m = 1  #Manufacturer sub index
g = 1  #GMSD index
s = 1  #State sub index
d = 38  #District sub index
i = 151 #Clinic sub index
t = 12  #Time sub index

clinic_breakpoints = [10,16,28,40,59,76,92,103,123,151,178,194,205,213,226,249,256,266,274,290,312,322,340,361,376,413,432,451,462,483,504,511,517,535,555,568,585,606]   #CLinic breakpoints for each districts
clinic_breakpoints = clinic_breakpoints[0:d]
i = clinic_breakpoints[d-1]

customers = list(range(1,j+1))
manufacturers = list(range(1,m+1))
gmsd = list(range(1,g+1))
svs = list(range(1,s+1))
dvs = list(range(1,d+1))
clinics = list(range(1,i+1))
time = list(range(1,t+1))

Lm = 0
Lg = 0
Ls = 0
Ld = 0
Li = 0
X = list(range(0,t-Li+1))

Ig_0 = 0
Is_0 = 0
Id_0 = 0
Ii_0 = 0

top_dvs = 15

In [3]:
########################### PARAMETERS ################################
l = GRB.INFINITY #large number for consistency constraints
fraction_storage = 0.5904 #Fraction of total capacity in cold chain points to be considered for COVID-19 vaccine
fraction_transport = 0.5904 #Fraction of total capacity in vehicles to be considered for COVID-19 vaccine

In [4]:
#Transportation cost
diesel_cost = 14
ftc_fac = 1
booking_cost = {
	"MG" : 40000,
	"GS" : 20000,
	"SD" : 12000*ftc_fac,
	"DI" : 5000*ftc_fac
}

#Distances
Dgm = [[1000]] #From M to G (confirm)
Dsg = [[550]] #From G to S (confirm)

#From S to D
df_Dds = pd.read_csv("Input_data/distances_sd.csv")
Dds = [[0 for D in range(d)] for S in range(s)]
for index in df_Dds.index:
    if (df_Dds['d'][index] > d):
        continue
    Dds[df_Dds['s'][index]-1][df_Dds['d'][index]-1] = df_Dds['Distance'][index]
    


#From D to I
df_Did = pd.read_csv("Input_data/distances_di.csv")
Did = [[0 for I in range(i)] for D in range(d)]
for index in df_Did.index:
    if (df_Did['d'][index] > d or df_Did['i'][index] > i):
        continue
    Did[df_Did['d'][index]-1][df_Did['i'][index]-1] = df_Did['Distance'][index]
    if(df_Did['d'][index]==1):
        if(df_Did['i'][index] not in range(0+1,clinic_breakpoints[df_Did['d'][index]-1]+1)):
            Did[df_Did['d'][index]-1][df_Did['i'][index]-1] = df_Did['Distance'][index]*1   
    else: 
        if(df_Did['i'][index] not in range(clinic_breakpoints[df_Did['d'][index]-2]+1,clinic_breakpoints[df_Did['d'][index]-1]+1)):
            Did[df_Did['d'][index]-1][df_Did['i'][index]-1] = df_Did['Distance'][index]*1

#Capacity of trucks
cap_veh_gm = round(fraction_transport*2932075) #Refrigerated van
cap_veh_sg = round(fraction_transport*2932075) #Refrigerated van
cap_veh_ds = round(fraction_transport*290880) #Insulated van
cap_veh_id = round(fraction_transport*290880) #Insulated van


#Final transportation costs
Kgmt = np.array([[[Dgm[M][G]*diesel_cost+booking_cost["MG"] for G in range(0,g)] for M in range(0,m)] for T in range(0,t)])
Ksgt = np.array([[[Dsg[G][S]*diesel_cost+booking_cost["GS"] for S in range(0,s)] for G in range(0,g)] for T in range(0,t)])
Kdst = np.array([[[Dds[S][D]*diesel_cost+booking_cost["SD"] for D in range(0,d)] for S in range(0,s)] for T in range(0,t)])
Kidt = np.array([[[Did[D][I]*diesel_cost+booking_cost["DI"] for I in range(0,i)] for D in range(0,d)] for T in range(0,t)])

In [5]:
########################## Finding top dvs from clinics ####################
top_dvs_arr = [[0 for D in range(top_dvs)] for I in clinics]
for I in clinics:
    temp = [(Did[D-1][I-1], D) for D in dvs]
    temp.sort()
    top_dvs_arr[I-1] = [temp[D][1] for D in range(top_dvs)]
    print(I)
    print(top_dvs_arr[I-1])
    print("")


1
[1, 27, 18, 16, 20, 36, 6, 29, 21, 4, 17, 9, 22, 5, 30]

2
[1, 27, 18, 16, 20, 36, 6, 29, 21, 4, 17, 9, 22, 5, 30]

3
[1, 20, 27, 36, 29, 16, 18, 6, 17, 21, 4, 5, 22, 9, 13]

4
[1, 27, 36, 20, 18, 16, 29, 21, 9, 6, 30, 4, 17, 23, 34]

5
[1, 27, 18, 16, 20, 36, 6, 29, 21, 4, 17, 9, 22, 5, 30]

6
[1, 27, 18, 36, 16, 20, 29, 21, 6, 9, 4, 17, 30, 22, 23]

7
[1, 36, 20, 27, 29, 18, 16, 21, 9, 17, 6, 30, 23, 34, 4]

8
[1, 18, 27, 16, 20, 36, 6, 29, 21, 4, 17, 9, 22, 5, 13]

9
[1, 27, 18, 16, 20, 36, 6, 29, 21, 4, 17, 9, 22, 5, 30]

10
[1, 27, 18, 16, 36, 20, 29, 21, 6, 9, 4, 17, 22, 30, 23]

11
[2, 14, 7, 26, 3, 11, 24, 37, 8, 25, 32, 15, 23, 28, 30]

12
[2, 14, 7, 26, 3, 11, 24, 37, 8, 25, 32, 15, 23, 28, 30]

13
[2, 3, 14, 7, 26, 11, 8, 24, 15, 28, 37, 25, 32, 23, 30]

14
[2, 14, 7, 26, 3, 11, 24, 25, 8, 37, 32, 15, 28, 23, 19]

15
[14, 2, 11, 26, 7, 24, 3, 25, 37, 32, 8, 23, 19, 30, 15]

16
[2, 14, 7, 26, 3, 11, 24, 37, 8, 25, 32, 15, 23, 28, 30]

17
[3, 15, 28, 2, 7, 8, 11, 14, 26, 25,

537
[12, 35, 31, 10, 37, 38, 33, 23, 26, 34, 8, 7, 14, 2, 30]

538
[31, 35, 12, 37, 10, 26, 33, 23, 34, 38, 8, 30, 7, 14, 2]

539
[35, 12, 31, 8, 10, 37, 38, 33, 26, 23, 34, 7, 15, 14, 2]

540
[35, 12, 31, 8, 10, 37, 38, 26, 33, 23, 34, 7, 15, 14, 2]

541
[35, 31, 12, 37, 10, 38, 26, 8, 33, 23, 34, 7, 30, 14, 2]

542
[35, 31, 12, 37, 10, 38, 26, 8, 33, 23, 34, 7, 30, 14, 2]

543
[35, 12, 31, 8, 10, 37, 38, 33, 26, 23, 7, 15, 34, 14, 2]

544
[35, 31, 12, 37, 10, 38, 26, 8, 33, 23, 34, 7, 30, 14, 2]

545
[35, 12, 31, 10, 37, 38, 26, 33, 8, 23, 34, 7, 14, 2, 30]

546
[35, 31, 12, 37, 10, 38, 26, 8, 33, 23, 34, 7, 30, 14, 2]

547
[35, 31, 12, 37, 10, 38, 26, 8, 33, 23, 34, 7, 30, 14, 2]

548
[31, 35, 12, 37, 10, 26, 8, 33, 23, 34, 38, 7, 30, 14, 2]

549
[35, 12, 31, 10, 37, 8, 38, 33, 26, 23, 34, 7, 15, 14, 2]

550
[35, 12, 31, 10, 38, 37, 8, 33, 26, 23, 34, 14, 7, 2, 15]

551
[35, 31, 12, 37, 10, 38, 26, 8, 33, 23, 34, 7, 30, 14, 2]

552
[35, 31, 12, 37, 10, 8, 26, 23, 38, 33, 7, 14, 34, 

In [6]:
#Shortage costs
SC = 1
Pjt = [[0 for J in range(j)] for T in range(t)]
for T in range(t):
    Pjt[T][0] = 75000*SC       #children
    Pjt[T][1] = 50000*SC       #adults
    Pjt[T][2] = 50000*SC       #elderly


#Clinical cost per unit of vaccine
Vj = 225

np.random.seed(0)
#Inventory holding costs
hgt = [[0.3 for G in range(g)] for T in range(t)]
hst = [[0.3 for S in range(s)] for T in range(t)]
hdt = np.random.normal(0.4,0,d*t).reshape(t,d)
hit = np.random.normal(0.4,0,i*t).reshape(t,i)

oc_fac = 1
#Ordering costs
Cgmt = [[[200000 for G in range(g)] for M in range(m)] for T in range(t)]
Csgt = [[[100000 for S in range(s)] for G in range(g)] for T in range(t)]
Cdst = [[[75000*oc_fac for D in range(d)] for S in range(s)] for T in range(t)]
Cidt = [[[15000*oc_fac for I in range(i)] for D in range(d)] for T in range(t)]

#Demand
wastage_factor = 0.5 #This value will depend on the vaccine, we are talking about.

In [7]:
#Fraction of demand
Fr_d = 1
#Fr_d = 0.5
#Fr_d = 0.75

df_demand = pd.read_csv("Input_data/weekly_demand.csv")
dijt = [[[0 for I in range(1,i+1)] for J in range(j)] for T in range(1,t+1)]
for index in df_demand.index:
    if (df_demand['i'][index] > i):
        break
    if(df_demand['t'][index]>t):
        continue
    dijt[df_demand['t'][index]-1][df_demand['j'][index]-1][df_demand['i'][index]-1] = round(df_demand['demand'][index]*Fr_d)

#Capacity of cold chain points
Bgt = [[round(fraction_storage*24545455) for G in range(g)] for T in range(t)]
Bst = [[round(fraction_storage*6818182) for S in range(s)] for T in range(t)]


df_bdt = pd.read_csv("Input_data/capacity_DVS.csv")
Bdt = [[0 for D in range(d)] for T in range(t)]
for index in df_bdt.index:
    if(df_bdt['d'][index] > d):
        break
    if(df_bdt['t'][index]>t):
        continue
    Bdt[df_bdt['t'][index]-1][df_bdt['d'][index]-1] = fraction_storage*df_bdt['Capacity'][index]


df_bit = pd.read_csv("Input_data/capacity_clinics.csv")
Bit = [[0 for I in range(i)] for T in range(t)]
for index in df_bit.index:
    if(df_bit['i'][index] > i):
        break
    if(df_bit['t'][index]>t):
        continue
    Bit[df_bit['t'][index]-1][df_bit['i'][index]-1] = fraction_storage*df_bit['Capacity'][index]



model = gp.Model('Vaccine_Distribution')

Academic license - for non-commercial use only - expires 2021-06-02
Using license file C:\Users\Ayush\gurobi.lic


In [8]:
#Production Capacity
# Bmt = [[round(350000*d/38) for M in range(m)] for T in range(t)]
Bmt = [[20000000 for M in range(m)] for T in range(t)]

#Average time required to administer the vaccine (minutes)
No = 5

#Number of medical personnel hours available (minutes)
Nit = [[3360 for I in range(i)] for T in range(t)]

#Hiring Cost of nurses
hc = 25000

#Firing Cost of nurses
fc = 10000

#Weekly wages of nurses
wg = 6175

#cap on trucks
cap_gm = 3
cap_sg = 3
cap_ds = 1
cap_id = 1

In [9]:
################### DECISION VARIABLES ##########################

#Inventory
Igt = model.addVars(time,gmsd,vtype=GRB.INTEGER, name="Igt")
# Igt = model.addVars(time,gmsd,vtype=GRB.CONTINUOUS, name="Igt")
Ist = model.addVars(time,svs,vtype=GRB.INTEGER, name="Ist")
# Ist = model.addVars(time,svs,vtype=GRB.CONTINUOUS, name="Ist")
Idt = model.addVars(time,dvs,vtype=GRB.INTEGER, name="Idt")
# Idt = model.addVars(time,dvs,vtype=GRB.CONTINUOUS, name="Idt")
Iit = model.addVars(time,clinics,vtype=GRB.INTEGER, name="Iit")
# Iit = model.addVars(time,clinics,vtype=GRB.CONTINUOUS, name="Iit")

#Quantity
Qgmt = model.addVars(time,manufacturers,gmsd,vtype=GRB.INTEGER, name="Qgmt")
# Qgmt = model.addVars(time,manufacturers,gmsd,vtype=GRB.CONTINUOUS, name="Qgmt")
Qsgt = model.addVars(time,gmsd,svs,vtype=GRB.INTEGER, name="Qsgt")
# Qsgt = model.addVars(time,gmsd,svs,vtype=GRB.CONTINUOUS, name="Qsgt")
Qdst = model.addVars(time,svs,dvs,vtype=GRB.INTEGER, name="Qdst")
# Qrst = model.addVars(time,svs,rvs,vtype=GRB.CONTINUOUS, name="Qrst")
Qidt = model.addVars(time,dvs,clinics,vtype=GRB.INTEGER, name="Qidt")
# Qidt = model.addVars(time,dvs,clinics,vtype=GRB.CONTINUOUS, name="Qidt")

#Shortage and Consumption
Sijt = model.addVars(time,customers,clinics,vtype=GRB.INTEGER, name="Sijt")
# Sijt = model.addVars(time,customers,clinics,vtype=GRB.CONTINUOUS, name="Sijt")
Wijt = model.addVars(time,customers,clinics,vtype=GRB.INTEGER, name="Wijt")
# Wijt = model.addVars(time,customers,clinics,vtype=GRB.CONTINUOUS, name="Wijt")

#Assignment Variables
Xgmt = model.addVars(time,manufacturers,gmsd,vtype=GRB.BINARY, name="Xgmt")
Xsgt = model.addVars(time,gmsd,svs,vtype=GRB.BINARY, name="Xsgt")
Xdst = model.addVars(time,svs,dvs,vtype=GRB.BINARY, name="Xdst")
Xidt = model.addVars(time,dvs,clinics,vtype=GRB.BINARY, name="Xidt")

#Number of trucks
Ngmt = model.addVars(time,manufacturers,gmsd,vtype=GRB.INTEGER, name="Ngmt")
Nsgt = model.addVars(time,gmsd,svs,vtype=GRB.INTEGER, name="Nsgt")
Ndst = model.addVars(time,svs,dvs,vtype=GRB.INTEGER, name="Ndst")
Nidt = model.addVars(time,dvs,clinics,vtype=GRB.INTEGER, name="Nidt")

#Nurses
N_nurses_it = model.addVars(time,clinics,vtype=GRB.INTEGER, name = "N_nurses_it")
H_nurses_it = model.addVars(time,clinics,vtype=GRB.INTEGER, name = "H_nurses_it")
F_nurses_it = model.addVars(time,clinics,vtype=GRB.INTEGER, name = "F_nurses_it")

In [10]:
############################# OBJECTIVE FUNCTION ###########################
transport_part = gp.quicksum(Kgmt[T-1][M-1][G-1]*Ngmt[T,M,G] for G in gmsd for M in manufacturers for T in time)
transport_part += gp.quicksum(Ksgt[T-1][G-1][S-1]*Nsgt[T,G,S] for S in svs for G in gmsd for T in time)
transport_part += gp.quicksum(Kdst[T-1][S-1][D-1]*Ndst[T,S,D] for D in dvs for S in svs for T in time)
transport_part += gp.quicksum(Kidt[T-1][D-1][I-1]*Nidt[T,D,I] for I in clinics for D in dvs for T in time)

inventory_part = gp.quicksum(hgt[T-1][G-1]*Igt[T,G] for G in gmsd for T in time)
inventory_part += gp.quicksum(hst[T-1][S-1]*Ist[T,S] for S in svs for T in time)
inventory_part += gp.quicksum(hdt[T-1][D-1]*Idt[T,D] for D in dvs for T in time)
inventory_part += gp.quicksum(hit[T-1][I-1]*Iit[T,I] for I in clinics for T in time)

shortage_part = gp.quicksum(Pjt[T-1][J-1]*Sijt[T,J,I] for J in customers for I in clinics for T in time)

consumption_part = gp.quicksum(Vj*Wijt[T,J,I] for J in customers for I in clinics for T in time) #Do we even need this?

ordering_part = gp.quicksum(Cgmt[T-1][M-1][G-1]*Xgmt[T,M,G] for G in gmsd for M in manufacturers for T in time)
ordering_part += gp.quicksum(Csgt[T-1][G-1][S-1]*Xsgt[T,G,S] for S in svs for G in gmsd for T in time)
ordering_part += gp.quicksum(Cdst[T-1][S-1][D-1]*Xdst[T,S,D] for D in dvs for S in svs for T in time)
ordering_part += gp.quicksum(Cidt[T-1][D-1][I-1]*Xidt[T,D,I] for I in clinics for D in dvs for T in time)

nurses_part = gp.quicksum(wg*N_nurses_it[T,I] for I in clinics for T in time)
nurses_part += gp.quicksum(hc*H_nurses_it[T,I] for I in clinics for T in time)
nurses_part += gp.quicksum(fc*F_nurses_it[T,I] for I in clinics for T in time)

model.setObjective(transport_part+inventory_part+shortage_part+consumption_part+ordering_part+nurses_part,GRB.MINIMIZE)


In [11]:
###################################### CONSTRAINTS ################################
#Pre-Processing
# pre_proc_1 = model.addConstrs(((Qidt[T,D,I] if D not in top_dvs_arr[I-1] else 0) == 0 for T in time for D in dvs for I in clinics),name = "pre_proc_1")
# pre_proc_2 = model.addConstrs(((Xidt[T,D,I] if D not in top_dvs_arr[I-1] else 0) == 0 for T in time for D in dvs for I in clinics),name = "pre_proc_2")
# pre_proc_3 = model.addConstrs(((Nidt[T,D,I] if D not in top_dvs_arr[I-1] else 0) == 0 for T in time for D in dvs for I in clinics),name = "pre_proc_3")


#Inventory Balance
gmsd_inventory = model.addConstrs(((Igt[T-1,G] if T>1 else Ig_0) + gp.quicksum((Qgmt[T-Lm,M,G] if T>Lm else 0) for M in manufacturers) 
                                    - Igt[T,G] == gp.quicksum(Qsgt[T,G,S] for S in svs) for G in gmsd for T in time),
                                     name="gmsd_inventory")
svs_inventory = model.addConstrs(((Ist[T-1,S] if T>1 else Is_0) + gp.quicksum((Qsgt[T-Lg,G,S] if T>Lg else 0) for G in gmsd) 
                                    - Ist[T,S] == gp.quicksum(Qdst[T,S,D] for D in dvs) for S in svs for T in time),
                                     name="svs_inventory")
dvs_inventory = model.addConstrs(((Idt[T-1,D] if T>1 else Id_0) + gp.quicksum((Qdst[T-Ls,S,D] if T>Ls else 0) for S in svs) 
                                    - Idt[T,D] == gp.quicksum(Qidt[T,D,I] for I in clinics) for D in dvs for T in time),
                                     name="dvs_inventory")
clinic_inventory = model.addConstrs(((Iit[T-1,I] if T>1 else Ii_0) + gp.quicksum((Qidt[T-Ld,D,I] if T>Ld else 0) for D in dvs) 
                                    - Iit[T,I] == gp.quicksum(Wijt[T,J,I] for J in customers) for I in clinics for T in time),
                                     name="clinic_inventory")

#Consumption by demand
consumption_demand = model.addConstrs((Wijt[T,J,I] <= dijt[T-1][J-1][I-1] for I in clinics for J in customers for T in time),
                                    name = "consumption_demand")

# Administration constraint
if(Li>0):
    administer_const = model.addConstrs((gp.quicksum(Wijt[T,J,I] for J in customers) <= (Iit[T-Li,I] if T>Li else Ii_0)
                                         for I in clinics for T in time),name = "administer_const")

#Consumption Balance
consumption_balance = model.addConstrs((Wijt[T,J,I] + Sijt[T,J,I] == dijt[T-1][J-1][I-1] for I in clinics for J in customers for T in time),
                                    name = "consumption_balance")

#Inventory Capacity constraints
gmsd_cap = model.addConstrs((Igt[T,G]<=Bgt[T-1][G-1] for G in gmsd for T in time),name = "gmsd_cap")
svs_cap = model.addConstrs((Ist[T,S]<=Bst[T-1][S-1] for S in svs for T in time),name = "svs_cap")
dvs_cap = model.addConstrs((Idt[T,D]<=Bdt[T-1][D-1] for D in dvs for T in time),name = "dvs_cap")
clinic_cap = model.addConstrs((Iit[T,I]<=Bit[T-1][I-1] for I in clinics for T in time),name = "clinic_cap")

#Production capacity constraints
production_cap = model.addConstrs((gp.quicksum(Qgmt[T,M,G] for G in gmsd)<= Bmt[T-1][M-1] for M in manufacturers for T in time)
                                ,name = "production_cap")

#Facility selection constraints
fac_sg = model.addConstrs((gp.quicksum(Xsgt[T,G,S] for G in gmsd)<=1 for S in svs for T in time),name = "fac_sg")
fac_ds = model.addConstrs((gp.quicksum(Xdst[T,S,D] for S in svs)<=1 for D in dvs for T in time),name = "fac_ds")
fac_id = model.addConstrs((gp.quicksum(Xidt[T,D,I] for D in dvs)<=1 for I in clinics for T in time),name = "fac_id")

#Constraints for consistency of q and X
# cons_1 = model.addConstrs((Xgmt[T,M,G]<=l*Qgmt[T,M,G] for G in gmsd for M in manufacturers for T in time),name = "cons_1")
cons_2 = model.addConstrs((Qgmt[T,M,G]<=cap_veh_gm*cap_gm*Xgmt[T,M,G] for G in gmsd for M in manufacturers for T in time),name = "cons_2")
# cons_3 = model.addConstrs((Xsgt[T,G,S]<=l*Qsgt[T,G,S] for S in svs for G in gmsd for T in time),name = "cons_3")
cons_4 = model.addConstrs((Qsgt[T,G,S]<=cap_veh_sg*cap_sg*Xsgt[T,G,S] for S in svs for G in gmsd for T in time),name = "cons_4")
# cons_5 = model.addConstrs((Xrst[T,S,R]<=l*Qrst[T,S,R] for R in rvs for S in svs for T in time),name = "cons_5")
cons_6 = model.addConstrs((Qdst[T,S,D]<=cap_veh_ds*cap_ds*Xdst[T,S,D] for D in dvs for S in svs for T in time),name = "cons_6")
# cons_9 = model.addConstrs((Xidt[T,D,I]<=l*Qidt[T,D,I] for I in clinics for D in dvs for T in time),name = "cons_9")
cons_10 = model.addConstrs((Qidt[T,D,I]<=cap_veh_id*cap_id*Xidt[T,D,I] for I in clinics for D in dvs for T in time),name = "cons_10")

# cons_1 = model.addConstrs((Xgmt[T,M,G]<=l*Qgmt[T,M,G] for G in gmsd for M in manufacturers for T in time),name = "cons_1")
# cons_2 = model.addConstrs((Qgmt[T,M,G]<=l*Xgmt[T,M,G] for G in gmsd for M in manufacturers for T in time),name = "cons_2")
# cons_3 = model.addConstrs((Xsgt[T,G,S]<=l*Qsgt[T,G,S] for S in svs for G in gmsd for T in time),name = "cons_3")
# cons_4 = model.addConstrs((Qsgt[T,G,S]<=l*Xsgt[T,G,S] for S in svs for G in gmsd for T in time),name = "cons_4")
# cons_5 = model.addConstrs((Xrst[T,S,R]<=l*Qrst[T,S,R] for R in rvs for S in svs for T in time),name = "cons_5")
# cons_6 = model.addConstrs((Qrst[T,S,R]<=l*Xrst[T,S,R] for R in rvs for S in svs for T in time),name = "cons_6")
# cons_7 = model.addConstrs((Xdrt[T,R,D]<=l*Qdrt[T,R,D] for D in dvs for R in rvs for T in time),name = "cons_7")
# cons_8 = model.addConstrs((Qdrt[T,R,D]<=l*Xdrt[T,R,D] for D in dvs for R in rvs for T in time),name = "cons_8")
# cons_9 = model.addConstrs((Xidt[T,D,I]<=l*Qidt[T,D,I] for I in clinics for D in dvs for T in time),name = "cons_9")
# cons_10 = model.addConstrs((Qidt[T,D,I]<=l*Xidt[T,D,I] for I in clinics for D in dvs for T in time),name = "cons_10")

#Number of trucks constraints
num_trucks_1 = model.addConstrs((Qgmt[T,M,G]/cap_veh_gm<=Ngmt[T,M,G] for G in gmsd for M in manufacturers for T in time),name = "num_trucks_1")
# num_trucks_2 = model.addConstrs((Ngmt[T,M,G]-Qgmt[T,M,G]/cap_veh_gm<=((cap_veh_gm-1)/cap_veh_gm) for G in gmsd for M in manufacturers for T in time),name = "num_trucks_2")
num_trucks_3 = model.addConstrs((Qsgt[T,G,S]/cap_veh_sg<=Nsgt[T,G,S] for S in svs for G in gmsd for T in time),name = "num_trucks_3")
# num_trucks_4 = model.addConstrs((Nsgt[T,G,S]-Qsgt[T,G,S]/cap_veh_sg<=((cap_veh_sg-1)/cap_veh_sg) for S in svs for G in gmsd for T in time),name = "num_trucks_4")
num_trucks_5 = model.addConstrs((Qdst[T,S,D]/cap_veh_ds<=Ndst[T,S,D] for D in dvs for S in svs for T in time),name = "num_trucks_5")
# num_trucks_6 = model.addConstrs((Nrst[T,S,R]-Qrst[T,S,R]/cap_veh_rs<=((cap_veh_rs-1)/cap_veh_rs) for R in rvs for S in svs for T in time),name = "num_trucks_6")
num_trucks_9 = model.addConstrs((Qidt[T,D,I]/cap_veh_id<=Nidt[T,D,I] for I in clinics for D in dvs for T in time),name = "num_trucks_9")
# num_trucks_10 = model.addConstrs((Nidt[T,D,I]-Qidt[T,D,I]/cap_veh_id<=((cap_veh_id-1)/cap_veh_id) for I in clinics for D in dvs for T in time),name = "num_trucks_10")

#Medical personnel availability constraints
med_constraint = model.addConstrs((gp.quicksum(No*Wijt[T,J,I] for J in customers)<=Nit[T-1][I-1]*N_nurses_it[T,I] for I in clinics for T in time),name = "med_constraint")

#Nurses Balance Constraint
nurses_constraint = model.addConstrs((N_nurses_it[T,I] == (N_nurses_it[T-1,I] if T>1 else 0)+ H_nurses_it[T,I] - F_nurses_it[T,I] for I in clinics for T in time),name = "nurses_constraint")

#Cap on trucks constraint
# trucks_constraint_gm = model.addConstrs((Ngmt[T,M,G]<=cap_gm for G in gmsd for M in manufacturers for T in time),name = "trucks_constraint_gm")
# trucks_constraint_sg = model.addConstrs((Nsgt[T,G,S]<=cap_sg for S in svs for G in gmsd for T in time),name = "trucks_constraint_sg")
# trucks_constraint_rs = model.addConstrs((Nrst[T,S,R]<=cap_rs for R in rvs for S in svs for T in time),name = "trucks_constraint_rs")
# trucks_constraint_dr = model.addConstrs((Ndrt[T,R,D]<=cap_dr for D in dvs for R in rvs for T in time),name = "trucks_constraint_dr")
# trucks_constraint_id = model.addConstrs((Nidt[T,D,I]<=cap_id for I in clinics for D in dvs for T in time),name = "trucks_constraint_id")



In [12]:
################################# Specifying intital solution ##############################

inv_df = pd.read_csv('inv.csv')
index = 0
for T in time:
    for G in gmsd:
        Igt[T,G].start = inv_df['Value'][index]
        index += 1
    for S in svs:
        Ist[T,S].start = inv_df['Value'][index]
        index += 1
    for D in svs:
        Idt[T,D].start = inv_df['Value'][index]
        index += 1
    for I in clinics:
        Iit[T,I].start = inv_df['Value'][index]
        index += 1
        


qty_df = pd.read_csv('qty.csv')
index = 0
for T in time:
    for M in manufacturers:
        for G in gmsd:
            Qgmt[T,M,G].start = qty_df['Value'][index]
            index += 1
    for G in gmsd:
        for S in svs:
            Qsgt[T,G,S].start = qty_df['Value'][index]
            index += 1
    for S in svs:
        for D in dvs:
            Qdst[T,S,D].start = qty_df['Value'][index]
            index += 1
    for D in dvs:
        for I in clinics:
            Qidt[T,D,I].start = qty_df['Value'][index]
            index += 1


con_df = pd.read_csv('cons.csv')
index = 0
for T in time:
    for I in clinics:
        for J in customers:
            Wijt[T,J,I].start = con_df['Value'][index]
            index += 1
            Sijt[T,J,I].start = con_df['Value'][index]
            index += 1


agn_df = pd.read_csv('agn.csv')
index = 0
for T in time:
    for M in manufacturers:
        for G in gmsd:
            Xgmt[T,M,G].start = agn_df['Value'][index]
            index += 1
    for G in gmsd:
        for S in svs:
            Xsgt[T,G,S].start = agn_df['Value'][index]
            index += 1
    for S in svs:
        for D in dvs:
            Xdst[T,S,D].start = agn_df['Value'][index]
            index += 1
    for D in dvs:
        for I in clinics:
            Xidt[T,D,I].start = agn_df['Value'][index]
            index += 1


trk_df = pd.read_csv('trk.csv')
index = 0
for T in time:
    for M in manufacturers:
        for G in gmsd:
            Ngmt[T,M,G].start = trk_df['Value'][index]
            index += 1
    for G in gmsd:
        for S in svs:
            Nsgt[T,G,S].start = trk_df['Value'][index]
            index += 1
    for S in svs:
        for D in dvs:
            Ndst[T,S,D].start = trk_df['Value'][index]
            index += 1
    for D in dvs:
        for I in clinics:
            Nidt[T,D,I].start = trk_df['Value'][index]
            index += 1


nur_df = pd.read_csv('nur.csv')
index = 0
for T in time:
    for I in clinics:
        N_nurses_it[T,I].start = nur_df['Value'][index]
        index += 1
        H_nurses_it[T,I].start = nur_df['Value'][index]
        index += 1
        F_nurses_it[T,I].start = nur_df['Value'][index]
        index += 1


In [13]:
################################# Solving the problem ##########################
# model.setParam('Method',3)
# model.setParam('Heuristics',1)
model.setParam('MIPGap',1e-6)
model.setParam("TimeLimit", 35000.0)
# model.setParam('MIPFocus',2)
model.optimize()

names = []
sol = []
for v in model.getVars():
    #print(v.varName,"=", v.x)
    names.append(v.varName)
    sol.append(v.x)
print("Done")

Changed value of parameter MIPGap to 1e-06
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 35000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 635064 rows, 903648 columns and 2105144 nonzeros
Model fingerprint: 0x73901475
Variable types: 0 continuous, 903648 integer (276816 binary)
Coefficient statistics:
  Matrix range     [6e-07, 5e+06]
  Objective range  [3e-01, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

User MIP start produced solution with objective 7.15227e+09 (1.27s)
Loaded user MIP start with objective 7.15227e+09
Processed MIP start in 1.36 seconds

Presolve removed 52471 rows and 37573 columns (presolve time = 5s) ...
Presolve removed 52471

  1351  1374 7.1221e+09  333 27653 7.1515e+09 7.1206e+09  0.43%   188 5488s
  1373  1402 7.1221e+09  340 27564 7.1515e+09 7.1206e+09  0.43%   189 5587s
  1401  1422 7.1221e+09  350 27480 7.1515e+09 7.1206e+09  0.43%   190 5692s
  1421  1448 7.1221e+09  357 27431 7.1515e+09 7.1206e+09  0.43%   193 5788s
  1447  1471 7.1221e+09  367 27347 7.1515e+09 7.1206e+09  0.43%   195 5885s
  1470  1491 7.1221e+09  378 27320 7.1515e+09 7.1206e+09  0.43%   199 5983s
  1490  1497 7.1222e+09  385 27105 7.1515e+09 7.1206e+09  0.43%   203 6101s
  1496  1509 7.1222e+09  386 27060 7.1515e+09 7.1206e+09  0.43%   209 6200s
  1508  1549 7.1222e+09  391 27273 7.1515e+09 7.1206e+09  0.43%   210 6282s
  1548  1573 7.1222e+09  400 27238 7.1515e+09 7.1206e+09  0.43%   211 6372s
  1572  1602 7.1222e+09  406 27116 7.1515e+09 7.1206e+09  0.43%   213 6468s
  1601  1614 7.1222e+09  411 27063 7.1515e+09 7.1206e+09  0.43%   214 6574s
  1613  1626 7.1222e+09  416 26976 7.1515e+09 7.1206e+09  0.43%   218 6691s
  1625  1636

  3653  3638 7.1224e+09  570 30241 7.1515e+09 7.1219e+09  0.41%   184 14962s
  3654  3638 7.1221e+09  283 30618 7.1515e+09 7.1220e+09  0.41%   184 14975s
  3655  3639 7.1220e+09   61 31465 7.1515e+09 7.1220e+09  0.41%   184 14985s
  3656  3640 7.1220e+09  209 32025 7.1515e+09 7.1220e+09  0.41%   184 14997s
  3657  3640 7.1222e+09  382 31960 7.1515e+09 7.1222e+09  0.41%   184 15010s
  3658  3641 7.1226e+09   94 31985 7.1515e+09 7.1226e+09  0.40%   184 15025s
  3659  3642 7.1231e+09  626 32011 7.1515e+09 7.1231e+09  0.40%   184 15039s
  3660  3642 7.1233e+09  471 32119 7.1515e+09 7.1233e+09  0.39%   184 15059s
  3661  3643 7.1233e+09  266 32407 7.1515e+09 7.1233e+09  0.39%   184 15073s
  3662  3644 7.1233e+09  425 32452 7.1515e+09 7.1233e+09  0.39%   183 15126s
H 3662  3461                    7.151121e+09 7.1233e+09  0.39%   183 15217s
  3664  3462 7.1234e+09  341 32931 7.1511e+09 7.1234e+09  0.39%   183 15239s
  3665  3463 7.1234e+09  668 33593 7.1511e+09 7.1234e+09  0.39%   183 15254s


  3760  3526 7.1326e+09  471 39336 7.1511e+09 7.1326e+09  0.26%   179 17582s
  3761  3527 7.1327e+09  266 39431 7.1511e+09 7.1327e+09  0.26%   179 17606s
  3762  3527 7.1327e+09  425 39510 7.1511e+09 7.1327e+09  0.26%   179 17632s
  3763  3528 7.1327e+09  980 39664 7.1511e+09 7.1327e+09  0.26%   179 17652s
  3764  3529 7.1327e+09  341 39722 7.1511e+09 7.1327e+09  0.26%   178 17673s
  3765  3529 7.1327e+09  668 39932 7.1511e+09 7.1327e+09  0.26%   178 17702s
  3766  3530 7.1327e+09 1202 40096 7.1511e+09 7.1327e+09  0.26%   178 17731s
  3767  3531 7.1328e+09  900 40298 7.1511e+09 7.1328e+09  0.26%   178 17756s
  3768  3531 7.1328e+09  914 40503 7.1511e+09 7.1328e+09  0.26%   178 17843s
  3769  3532 7.1328e+09   77 40849 7.1511e+09 7.1328e+09  0.26%   178 17948s
  3770  3533 7.1328e+09   52 41219 7.1511e+09 7.1328e+09  0.26%   178 17991s
  3771  3533 7.1328e+09  430 41199 7.1511e+09 7.1328e+09  0.26%   178 18034s
  3772  3534 7.1328e+09  447 41051 7.1511e+09 7.1328e+09  0.26%   178 18077s

  3867  3597 7.1350e+09  900 42456 7.1511e+09 7.1350e+09  0.22%   174 30806s
  3868  3598 7.1350e+09  914 42554 7.1511e+09 7.1350e+09  0.22%   174 30943s
  3869  3599 7.1350e+09   77 42564 7.1511e+09 7.1350e+09  0.22%   174 31079s
  3870  3599 7.1350e+09   52 42767 7.1511e+09 7.1350e+09  0.22%   174 31197s
  3871  3600 7.1350e+09  430 42763 7.1511e+09 7.1350e+09  0.22%   174 31327s
  3872  3601 7.1350e+09  447 42816 7.1511e+09 7.1350e+09  0.22%   174 31423s
  3873  3601 7.1350e+09  240 42811 7.1511e+09 7.1350e+09  0.22%   173 31537s
  3874  3602 7.1350e+09  225 42811 7.1511e+09 7.1350e+09  0.22%   173 31659s
  3876  3605 7.1350e+09 1215 32378 7.1511e+09 7.1350e+09  0.22%   368 31707s
  3878  3606 7.1350e+09  470 14240 7.1511e+09 7.1350e+09  0.22%   367 31804s
  3879  3607 7.1350e+09  296 14237 7.1511e+09 7.1350e+09  0.22%   367 31889s
  3880  3608 7.1350e+09  457 14334 7.1511e+09 7.1350e+09  0.22%   367 31895s
  3881  3608 7.1350e+09  348 14451 7.1511e+09 7.1350e+09  0.22%   367 31901s

In [ ]:
# ###########################Code for full excel sheet results generation##########################
workbook = xlsxwriter.Workbook("no-pre"+str(ftc_fac)+' -- '+str(oc_fac)+'districts-'+str(d)+' manuf-'+ str(Bmt[0][0])+'-full-sheet-inventory-ini-'+'.xlsx' )
worksheet = workbook.add_worksheet()
merge_format = workbook.add_format({
    'bold': 1,
    'border': 1, 
    'align': 'center',
    'valign': 'vcenter'})

worksheet.merge_range('A1:B1', 'Inventory', merge_format)
worksheet.merge_range('C1:D1', 'Consumption', merge_format)
worksheet.merge_range('E1:F1', 'Shortage', merge_format)
worksheet.merge_range('G1:H1', 'Delivery from M to G', merge_format)
worksheet.merge_range('I1:J1', 'X from M to G', merge_format)
worksheet.merge_range('K1:L1', 'No of Trucks from M to G', merge_format)
worksheet.merge_range('M1:N1', 'Distance from M to G', merge_format)
worksheet.merge_range('O1:P1', 'Delivery from G to S', merge_format)
worksheet.merge_range('Q1:R1', 'X from G to S', merge_format)
worksheet.merge_range('S1:T1', 'No of Trucks from G to S', merge_format)
worksheet.merge_range('U1:V1', 'Distance from G to S', merge_format)
worksheet.merge_range('W1:X1', 'Delivery from S to D', merge_format)
worksheet.merge_range('Y1:Z1', 'X from S to D', merge_format)
worksheet.merge_range('AA1:AB1', 'No of Trucks from S to D', merge_format)
worksheet.merge_range('AC1:AD1', 'Distance from S to D', merge_format)
worksheet.merge_range('AE1:AF1', 'Delivery from D to I', merge_format)
worksheet.merge_range('AG1:AH1', 'X from D to I', merge_format)
worksheet.merge_range('AI1:AJ1', 'No of Trucks from D to I', merge_format)
worksheet.merge_range('AK1:AL1', 'Distance from D to I', merge_format)


#Inventory
row = 1
for T in time:
    for G in gmsd:
        worksheet.write(row,0,Igt[T,G].varName)
        worksheet.write(row,1,round(Igt[T,G].x))
        row += 1

for T in time:
    for S in svs:
        worksheet.write(row,0,Ist[T,S].varName)
        worksheet.write(row,1,round(Ist[T,S].x))
        row += 1
        
for T in time:
    for D in dvs:
        worksheet.write(row,0,Idt[T,D].varName)
        worksheet.write(row,1,round(Idt[T,D].x))
        row += 1

for T in time:
    for I in clinics:
        worksheet.write(row,0,Iit[T,I].varName)
        worksheet.write(row,1,round(Iit[T,I].x))
        row += 1

#Qgmt, Xgmt, Ngmt
row = 1
for T in time:
    for M in manufacturers:
        for G in gmsd:
            worksheet.write(row,6,Qgmt[T,M,G].varName)
            worksheet.write(row,7,round(Qgmt[T,M,G].x))
            worksheet.write(row,8,Xgmt[T,M,G].varName)
            worksheet.write(row,9,round(Xgmt[T,M,G].x))
            worksheet.write(row,10,Ngmt[T,M,G].varName)
            worksheet.write(row,11,round(Ngmt[T,M,G].x))
            row += 1
#Dgm
row = 1
for M in manufacturers:
    for G in gmsd:
        variable = "Dgm["+str(M)+","+str(G)+"]"
        worksheet.write(row,12,variable)
        worksheet.write(row,13,Dgm[M-1][G-1])
        row += 1
        

#Qsgt,Xsgt,Nsgt
row = 1
for T in time:
    for G in gmsd:
        for S in svs:
            worksheet.write(row,14,Qsgt[T,G,S].varName)
            worksheet.write(row,15,round(Qsgt[T,G,S].x))
            worksheet.write(row,16,Xsgt[T,G,S].varName)
            worksheet.write(row,17,round(Xsgt[T,G,S].x))
            worksheet.write(row,18,Nsgt[T,G,S].varName)
            worksheet.write(row,19,round(Nsgt[T,G,S].x))
            row += 1
#Dsg
row = 1
for G in gmsd:
    for S in svs:
        variable = "Dsg["+str(G)+","+str(S)+"]"
        worksheet.write(row,20,variable)
        worksheet.write(row,21,Dsg[G-1][S-1])
        row += 1

#Qrst,Xrst,Nrst
row = 1
for T in time:
    for S in svs:
        for D in dvs:
            worksheet.write(row,22,Qdst[T,S,D].varName)
            worksheet.write(row,23,round(Qdst[T,S,D].x))
            worksheet.write(row,24,Xdst[T,S,D].varName)
            worksheet.write(row,25,round(Xdst[T,S,D].x))
            worksheet.write(row,26,Ndst[T,S,D].varName)
            worksheet.write(row,27,round(Ndst[T,S,D].x))
            row += 1
#Dds
row = 1
for S in svs:
    for D in dvs:
        variable = "Dds["+str(S)+","+str(D)+"]"
        worksheet.write(row,28,variable)
        worksheet.write(row,29,Dds[S-1][D-1])
        row += 1

        
#Qidt,Xidt,Nidt
row = 1
for T in time:
    for D in dvs:
        for I in clinics:
            worksheet.write(row,30,Qidt[T,D,I].varName)
            worksheet.write(row,31,round(Qidt[T,D,I].x))
            worksheet.write(row,32,Xidt[T,D,I].varName)
            worksheet.write(row,33,round(Xidt[T,D,I].x))
            worksheet.write(row,34,Nidt[T,D,I].varName)
            worksheet.write(row,35,round(Nidt[T,D,I].x))
            row += 1
#Did
row = 1
for D in dvs:
    for I in clinics:
        variable = "Did["+str(D)+","+str(I)+"]"
        worksheet.write(row,36,variable)
        worksheet.write(row,37,Did[D-1][I-1])
        row += 1

#Shortage
row = 1
for T in time:
    for J in customers:
        for I in clinics:
            worksheet.write(row,4,Sijt[T,J,I].varName)
            worksheet.write(row,5,round(Sijt[T,J,I].x))
            row += 1


#Consumption
row = 1
for T in time:
    for J in customers:
        for I in clinics:
            worksheet.write(row,2,Wijt[T,J,I].varName)
            worksheet.write(row,3,round(Wijt[T,J,I].x))
            row += 1



workbook.close()

In [ ]:
############################################ Summary #####################################################

############## transport part ###############
no_of_times_M = ""
cost_M = ""
total_cost_M = 0
for M in manufacturers:
    number = 0
    cost = 0
    for G in gmsd:
        for T in time:
            number =  number + Xgmt[T,M,G].x
            cost = cost + Kgmt[T-1][M-1][G-1]*Ngmt[T,M,G].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_M==""):
            no_of_times_M += (str(M)+"("+str(number)+" times"+")")
            cost_M += (str(M)+"("+str(cost)+" Rs"+")")
        else:
            no_of_times_M += (", "+str(M)+"("+str(number)+" times"+")")
            cost_M += (", "+str(M)+"("+str(cost)+" Rs"+")")
        total_cost_M += cost


no_of_times_G = ""
cost_G = ""
total_cost_G = 0
for G in gmsd:
    number = 0
    cost = 0
    for S in svs:
        for T in time:
            number =  number + Xsgt[T,G,S].x
            cost = cost + Ksgt[T-1][G-1][S-1]*Nsgt[T,G,S].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_G==""):
            no_of_times_G += (str(G)+"("+str(number)+" times"+")")
            cost_G += (str(G)+"("+str(cost)+" Rs"+")")
        else:
            no_of_times_G += (", "+str(G)+"("+str(number)+" times"+")")
            cost_G += (", "+str(G)+"("+str(cost)+" Rs"+")")
        total_cost_G += cost


no_of_times_S = ""
cost_S = ""
total_cost_S = 0
for S in svs:
    number = 0
    cost = 0
    for R in rvs:
        for T in time:
            number =  number + Xrst[T,S,R].x
            cost = cost + Krst[T-1][S-1][R-1]*Nrst[T,S,R].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_S==""):
            no_of_times_S += (str(S)+"("+str(number)+" times"+")")
            cost_S += (str(S)+"("+str(cost)+" Rs"+")")
        else:
            no_of_times_S += (", "+str(S)+"("+str(number)+" times"+")")
            cost_S += (", "+str(S)+"("+str(cost)+" Rs"+")")
        total_cost_S += cost


no_of_times_R = ""
cost_R = ""
total_cost_R = 0
for R in rvs:
    number = 0
    cost = 0
    for D in dvs:
        for T in time:
            number =  number + Xdrt[T,R,D].x
            cost = cost + Kdrt[T-1][R-1][D-1]*Ndrt[T,R,D].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_R==""):
            no_of_times_R += (str(R)+"("+str(number)+" times"+")")
            cost_R += (str(R)+"("+str(cost)+" Rs"+")")
        else:
            no_of_times_R += (", "+str(R)+"("+str(number)+" times"+")")
            cost_R += (", "+str(R)+"("+str(cost)+" Rs"+")")
        total_cost_R += cost

no_of_times_D = ""
cost_D = ""
total_cost_D = 0
for D in dvs:
    number = 0
    cost = 0
    for I in clinics:
        for T in time:
            number =  number + Xidt[T,D,I].x
            cost = cost + Kidt[T-1][D-1][I-1]*Nidt[T,D,I].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_D==""):
            no_of_times_D += (str(D)+"("+str(number)+" times"+")")
            cost_D += (str(D)+"("+str(cost)+" Rs"+")")
        else:
            no_of_times_D += (", "+str(D)+"("+str(number)+" times"+")")
            cost_D += (", "+str(D)+"("+str(cost)+" Rs"+")")
        total_cost_D += cost


transport_summary = {
    "From":["M->G","G->S","S->R","R->D","D->I"],
    "Number of times transport occurs over the entire planning horizon":[no_of_times_M,no_of_times_G,no_of_times_S,no_of_times_R,no_of_times_D],
    "Cost Incurred":[cost_M,cost_G,cost_S,cost_R,cost_D],
    "Total Cost":[total_cost_M,total_cost_G,total_cost_S,total_cost_R,total_cost_D]
}


################ ordering part ##############

average_quantity_M = ""
cost_M = ""
total_cost_M = 0
for M in manufacturers:
    number = 0
    cost = 0
    quantity = 0
    for G in gmsd:
        for T in time:
            number =  number + Xgmt[T,M,G].x
            cost = cost + Cgmt[T-1][M-1][G-1]*Xgmt[T,M,G].x
            quantity = quantity + Qgmt[T,M,G].x
    cost = round(cost)
    if(number!=0):
        average_quantity = quantity/number
        if(average_quantity_M==""):
            average_quantity_M += (str(M)+"("+str(round(average_quantity))+" units"+")")
            cost_M += (str(M)+"("+str(cost)+" Rs"+")")
        else:
            average_quantity_M += (", "+str(M)+"("+str(round(average_quantity))+" units"+")")
            cost_M += (", "+str(M)+"("+str(cost)+" Rs"+")")
        total_cost_M += cost


average_quantity_G = ""
cost_G = ""
total_cost_G = 0
for G in gmsd:
    number = 0
    cost = 0
    quantity = 0
    for S in svs:
        for T in time:
            number =  number + Xsgt[T,G,S].x
            cost = cost + Csgt[T-1][G-1][S-1]*Xsgt[T,G,S].x
            quantity = quantity + Qsgt[T,G,S].x
    cost = round(cost)
    if(number!=0):
        average_quantity = quantity/number
        if(average_quantity_G==""):
            average_quantity_G += (str(G)+"("+str(round(average_quantity))+" units"+")")
            cost_G += (str(G)+"("+str(cost)+" Rs"+")")
        else:
            average_quantity_G += (", "+str(G)+"("+str(round(average_quantity))+" units"+")")
            cost_G += (", "+str(G)+"("+str(cost)+" Rs"+")")
        total_cost_G += cost


average_quantity_S = ""
cost_S = ""
total_cost_S = 0
for S in svs:
    number = 0
    cost = 0
    quantity = 0
    for R in rvs:
        for T in time:
            number =  number + Xrst[T,S,R].x
            cost = cost + Crst[T-1][S-1][R-1]*Xrst[T,S,R].x
            quantity = quantity + Qrst[T,S,R].x
    cost = round(cost)
    if(number!=0):
        average_quantity = quantity/number
        if(average_quantity_S==""):
            average_quantity_S += (str(S)+"("+str(round(average_quantity))+" units"+")")
            cost_S += (str(S)+"("+str(cost)+" Rs"+")")
        else:
            average_quantity_S += (", "+str(S)+"("+str(round(average_quantity))+" units"+")")
            cost_S += (", "+str(S)+"("+str(cost)+" Rs"+")")
        total_cost_S += cost


average_quantity_R = ""
cost_R = ""
total_cost_R = 0
for R in rvs:
    number = 0
    cost = 0
    quantity = 0
    for D in dvs:
        for T in time:
            number =  number + Xdrt[T,R,D].x
            cost = cost + Cdrt[T-1][R-1][D-1]*Xdrt[T,R,D].x
            quantity = quantity + Qdrt[T,R,D].x
    cost = round(cost)
    if(number!=0):
        average_quantity = quantity/number
        if(average_quantity_R==""):
            average_quantity_R += (str(R)+"("+str(round(average_quantity))+" units"+")")
            cost_R += (str(R)+"("+str(cost)+" Rs"+")")
        else:
            average_quantity_R += (", "+str(R)+"("+str(round((average_quantity)))+" units"+")")
            cost_R += (", "+str(R)+"("+str(cost)+" Rs"+")")
        total_cost_R += cost

average_quantity_D = ""
cost_D = ""
total_cost_D = 0
for D in dvs:
    number = 0
    cost = 0
    quantity = 0
    for I in clinics:
        for T in time:
            number =  number + Xidt[T,D,I].x
            cost = cost + Cidt[T-1][D-1][I-1]*Xidt[T,D,I].x
            quantity = quantity + Qidt[T,D,I].x
    cost = round(cost)
    if(number!=0):
        average_quantity = quantity/number
        if(average_quantity_D==""):
            average_quantity_D += (str(D)+"("+str(round(average_quantity))+" units"+")")
            cost_D += (str(D)+"("+str(cost)+" Rs"+")")
        else:
            average_quantity_D += (", "+str(D)+"("+str(round(average_quantity))+" units"+")")
            cost_D += (", "+str(D)+"("+str(cost)+" Rs"+")")
        total_cost_D += cost


ordering_summary = {
    "From":["M->G","G->S","S->R","R->D","D->I"],
    "Average quantities ordered over the entire planning horizon":[average_quantity_M,average_quantity_G,average_quantity_S,average_quantity_R,average_quantity_D],
    "Cost Incurred":[cost_M,cost_G,cost_S,cost_R,cost_D],
    "Total Cost":[total_cost_M,total_cost_G,total_cost_S,total_cost_R,total_cost_D]
}

################ inventory part #############

no_of_times_G = ""
avg_inv_G = ""
cost_G = ""
total_cost_G = 0
for G in gmsd:
    number = 0
    cost = 0
    total_inventory_G = 0
    for T in time:
        if(Igt[T,G].x>0):
            number += 1
            total_inventory_G += Igt[T,G].x
            cost += hgt[T-1][G-1]*Igt[T,G].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_G==""):
            no_of_times_G += (str(G)+"("+str(number)+" times"+")")
            cost_G += (str(G)+"("+str(cost)+" Rs"+")")
            avg_inv_G += (str(G)+"("+str(total_inventory_G/number)+" units"+")")

        else:
            no_of_times_G += (", "+str(G)+"("+str(number)+" times"+")")
            cost_G += (", "+str(G)+"("+str(cost)+" Rs"+")")
            if(number!=0):
                avg_inv_G += (", "+str(G)+"("+str(total_inventory_G/number)+" units"+")")
            else:
                avg_inv_G += (", "+str(G)+"("+str(0)+" units"+")")
    total_cost_G += cost
        

no_of_times_S = ""
avg_inv_S = ""
cost_S = ""
total_cost_S = 0
for S in svs:
    number = 0
    cost = 0
    total_inventory_S = 0
    for T in time:
        if(Ist[T,S].x>0):
            number += 1
            total_inventory_S += Ist[T,S].x
            cost += hst[T-1][S-1]*Ist[T,S].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_S==""):
            no_of_times_S += (str(S)+"("+str(number)+" times"+")")
            cost_S += (str(S)+"("+str(cost)+" Rs"+")")
            avg_inv_S += (str(S)+"("+str(total_inventory_S/number)+" units"+")")
    
        else:
            no_of_times_S += (", "+str(S)+"("+str(number)+" times"+")")
            cost_S += (", "+str(S)+"("+str(cost)+" Rs"+")")
            if(number!=0):
                avg_inv_S += (", "+str(S)+"("+str(total_inventory_S/number)+" units"+")") 
            else: 
                avg_inv_S += (", "+str(S)+"("+str(0)+" units"+")") 
    total_cost_S += cost




no_of_times_D = ""
avg_inv_D = ""
cost_D = ""
total_cost_D = 0
for D in dvs:
    number = 0
    cost = 0
    total_inventory_D = 0
    for T in time:
        if(Idt[T,D].x>0):
            number += 1
            total_inventory_D += Idt[T,D].x
            cost += hdt[T-1][D-1]*Idt[T,D].x
    cost = round(cost)
    if(number!=0):
        if(no_of_times_D==""):
                no_of_times_D += (str(D)+"("+str(number)+" times"+")")
                cost_D += (str(D)+"("+str(cost)+" Rs"+")")
                if(number!=0):
                    avg_inv_D += (str(D)+"("+str(total_inventory_D/number)+" units"+")")
                else:
                    avg_inv_D += (str(D)+"("+str(0)+" units"+")")
        else:
            no_of_times_D += (", "+str(D)+"("+str(number)+" times"+")")
            cost_D += (", "+str(D)+"("+str(cost)+" Rs"+")")
            if(number!=0):
                avg_inv_D += (", "+str(D)+"("+str(total_inventory_D/number)+" units"+")")  
            else:
                avg_inv_D += (", "+str(D)+"("+str(0)+" units"+")")
    total_cost_D += cost


I_s = 1
total_cost_I = 0
no_of_times_I = ""
D = 1
cost_I = ""
avg_inv_I = ""
for I_b in clinic_breakpoints:
    number_average_district = 0
    avg_inv_avg_district = 0
    cost_avg_district = 0
    for I in range(I_s,I_b+1):
        number = 0
        cost = 0
        total_inventory_I = 0
        for T in time:
            if(Iit[T,I].x>0):
                number += 1
                total_inventory_I += Iit[T,I].x
                cost += hit[T-1][I-1]*Iit[T,I].x
        number_average_district += number
        if(number!=0):
            avg_inv_avg_district += total_inventory_I/number
        else:
            avg_inv_avg_district += 0
        cost_avg_district += cost
        total_cost_I += cost

    number_average_district /= (I_b-I_s+1)
    number_average_district = round(number_average_district)

    avg_inv_avg_district /= (I_b-I_s+1)
    avg_inv_avg_district = round(avg_inv_avg_district)

    cost_avg_district /= (I_b-I_s+1)

    if(no_of_times_I==""):
            no_of_times_I += (str(D)+"("+str(number_average_district)+" times"+")")
            cost_I += (str(D)+"("+str(round(cost_avg_district))+" Rs"+")")
            avg_inv_I += (str(D)+"("+str(round(avg_inv_avg_district))+" units"+")")
    else:
        no_of_times_I += (", "+str(D)+"("+str(number_average_district)+" times"+")")
        cost_I += (", "+str(D)+"("+str(round(cost_avg_district))+" Rs"+")")
        avg_inv_I += (", "+str(D)+"("+str(round(avg_inv_avg_district))+" units"+")") 

    D = D+1
    I_s = I_b+1

inventory_summary = {
    "CCP": ["G","S","R","D","I"],
    "Number of times Inventory is non zero":[no_of_times_G,no_of_times_S,no_of_times_R,no_of_times_D,no_of_times_I],
    "Average Inventory carried":[avg_inv_G,avg_inv_S,avg_inv_R,avg_inv_D,avg_inv_I],
    "Cost Incurred":[cost_G,cost_S,cost_R,cost_D,cost_I],
    "Total Cost":[total_cost_G,total_cost_S,total_cost_R,total_cost_D,total_cost_I]
}

########################### Shortages summary ##############################


I_s = 1
num1 = 0
costs_list = []
average_costs_list = []
number_of_clinics = []
shortages_list = []
count = 0
clinic_num = []
total_shortage = []
for num in clinic_breakpoints:
    cost = [0 for J in range(j)]
    fraction_of_shortages = [0 for J in range(j)]
    num_clinics = num - num1
    count += 1
    total_demand = [0 for J in range(j)]
    for x in range(num_clinics):
        I = I_s + x
        for J in customers:
            for T in time:
                total_demand[J-1] += dijt[T-1][J-1][I-1]
                if(Sijt[T,J,I].x!=0):
                    cost[J-1] += Pjt[T-1][J-1]*Sijt[T,J,I].x
                    fraction_of_shortages[J-1]+=Sijt[T,J,I].x
    total_shortages = sum(fraction_of_shortages)
    for J in customers:
        fraction_of_shortages[J-1] = fraction_of_shortages[J-1]*100/total_demand[J-1]    
        
    avg_cost = [cost[J-1]/num_clinics for J in customers]
    I_s += num_clinics
    num1 = num
    shortage_string = ""
    cost_string = ""
    avg_cost_string = ""
    for J in customers:
        cost_string += str(J)+"("+str(cost[J-1])+"), "
        avg_cost_string += str(J)+"("+str(avg_cost[J-1])+"), "
        shortage_string += str(J)+"("+str(fraction_of_shortages[J-1])+"), "

    costs_list.append(cost_string)
    average_costs_list.append(avg_cost_string)
    number_of_clinics.append(num_clinics)
    shortages_list.append(shortage_string)
    clinic_num.append(count)
    total_shortage.append(total_shortages)

shortage_summary = {
    "District Number": clinic_num,
    "Number of clinics": number_of_clinics,
    "Total shortages": total_shortage,
    "Total shortage cost Incurred": costs_list,
    "Percentage(group-wise) of shortages": shortages_list,
    "Average shortage cost incurred per clinic":average_costs_list
}

################################## Nurses part ####################################
I_s = 1
num1 = 0
avg_number_list = []
avg_hiring_list = []
avg_firing_list = []
count = 0
clinic_num = []
for num in clinic_breakpoints:
    num_clinics = num - num1
    count += 1
    n_nurses = 0
    n_hiring = 0
    n_firing = 0
    for x in range(num_clinics):
        I = I_s + x
        for T in time:
            n_nurses += N_nurses_it[T,I].x
            n_hiring += H_nurses_it[T,I].x
            n_firing += F_nurses_it[T,I].x
    
    I_s += num_clinics
    num1 = num
    avg_number_list.append(n_nurses/(num_clinics*t))
    avg_hiring_list.append(n_hiring/(num_clinics*t))
    avg_firing_list.append(n_firing/(num_clinics*t))
    clinic_num.append(count)


nurses_summary = {
"District Number": clinic_num,
"Avg number of nurses per week": avg_number_list,
"Avg number of hired nurses per week": avg_hiring_list,
"Avg number of fired nurses per week": avg_firing_list
}

transport_df = pd.DataFrame.from_dict(transport_summary)
inventory_df = pd.DataFrame.from_dict(inventory_summary)
ordering_df = pd.DataFrame.from_dict(ordering_summary)
shortage_df = pd.DataFrame.from_dict(shortage_summary)
nurses_df = pd.DataFrame.from_dict(nurses_summary)

###########################################Compiled Results##################################################
writer = pd.ExcelWriter('with_districts-'+str(d)+' fraction-'+str(fraction_storage)+' manuf-'+ str(Bmt[0][0]) +'-without-x-inventory-ini-'+str(Ii_0)+"---"+str(Lm)+"-"+str(Lg)+"-"+str(Ls)+"-"+str(Lr)+"-"+str(Ld)+"-"+str(Li)+'shortage-'+str(SC)+'.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('Compiled')
writer.sheets['Compiled'] = worksheet
worksheet.write(0,0,"Transport part")
transport_df.to_excel(writer,sheet_name='Compiled',startrow=2 , startcol=0)

x = 4 + len(transport_df.index)
worksheet.write(x,0,"Inventory part")
x += 2
inventory_df.to_excel(writer,sheet_name='Compiled',startrow=x , startcol=0)

x += 4 + len(inventory_df.index)
worksheet.write(x,0,"Ordering part")
x += 2
ordering_df.to_excel(writer,sheet_name='Compiled',startrow=x , startcol=0)

x += 4 + len(ordering_df.index)
worksheet.write(x,0,"Shortage part")
x += 2
shortage_df.to_excel(writer,sheet_name='Compiled',startrow=x , startcol=0)

x += 4 + len(shortage_df.index)
worksheet.write(x,0,"Nurses part")
x += 2
nurses_df.to_excel(writer,sheet_name='Compiled',startrow=x , startcol=0)
workbook.close()

################################################# The End ######################################################### 

In [ ]:
dvss = []      
for I in clinics:
    temp = []
    for T in time:    
        for D in dvs:
            if(Xidt[T,D,I].x==1 and D not in top_dvs_arr[I-1]):
                    print(I)
                    print(D)
                    print(T)
                    print("")
            if(Xidt[T,D,I].x==1 and D not in temp):
                temp.append(D)
                
    dvss.append(temp)
clinic_breakpoints = [10,16,28,40,59,76,92,103,123,151,178,194,205,213,226,249,256,266,274,290,312,322,340,361,376,413,432,451,462,483,504,511,517,535,555,568,585,606]   #CLinic breakpoints for each districts
arr = []
clinic_no = 1 
dd = 1
for c in clinic_breakpoints:
    while(clinic_no<=c):
        arr.append(dd)
        clinic_no = clinic_no+1
    dd = dd+1 
arr = arr[:i]
dict_dvs = {
    "Clinincs": range(1,i+1),
    "DVSs":dvss,
    "original":arr,
    "Dist from 1": [Did[0][I-1] for I in clinics],
    "Dist from 2": [Did[1][I-1] for I in clinics],
    "Dist from 3": [Did[2][I-1] for I in clinics],
    "Dist from 4": [Did[3][I-1] for I in clinics],
    "Dist from 5": [Did[4][I-1] for I in clinics],
    "Dist from 6": [Did[5][I-1] for I in clinics],
    "Dist from 7": [Did[6][I-1] for I in clinics],
    "Dist from 8": [Did[7][I-1] for I in clinics],
    "Dist from 9": [Did[8][I-1] for I in clinics],
    "Dist from 10": [Did[9][I-1] for I in clinics],
    "Dist from 11": [Did[10][I-1] for I in clinics],
    "Dist from 12": [Did[11][I-1] for I in clinics],
    "Dist from 13": [Did[12][I-1] for I in clinics],
    "Dist from 14": [Did[13][I-1] for I in clinics],
    "Dist from 15": [Did[14][I-1] for I in clinics],
    "Dist from 16": [Did[15][I-1] for I in clinics],
    "Dist from 17": [Did[16][I-1] for I in clinics],
    "Dist from 18": [Did[17][I-1] for I in clinics],
    "Dist from 19": [Did[18][I-1] for I in clinics],
    "Dist from 20": [Did[19][I-1] for I in clinics],
    "Dist from 21": [Did[20][I-1] for I in clinics],
    "Dist from 22": [Did[21][I-1] for I in clinics],
    "Dist from 23": [Did[22][I-1] for I in clinics],
    "Dist from 24": [Did[23][I-1] for I in clinics],
    "Dist from 25": [Did[24][I-1] for I in clinics],
    "Dist from 26": [Did[25][I-1] for I in clinics],
    "Dist from 27": [Did[26][I-1] for I in clinics],
    "Dist from 28": [Did[27][I-1] for I in clinics],
    "Dist from 29": [Did[28][I-1] for I in clinics],
    "Dist from 30": [Did[29][I-1] for I in clinics],
    "Dist from 31": [Did[30][I-1] for I in clinics],
    "Dist from 32": [Did[31][I-1] for I in clinics],
    "Dist from 33": [Did[32][I-1] for I in clinics],
    "Dist from 34": [Did[33][I-1] for I in clinics],
    "Dist from 35": [Did[34][I-1] for I in clinics],
    "Dist from 36": [Did[35][I-1] for I in clinics],
    "Dist from 37": [Did[36][I-1] for I in clinics],
    "Dist from 38": [Did[37][I-1] for I in clinics],
}
dvs_df = pd.DataFrame.from_dict(dict_dvs)
# print(dvs_df)

          

# print(dvss)
# print(rvs_df)
writer = pd.ExcelWriter('no-pre-temp'+str(d)+'.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('Compiled')
writer.sheets['Compiled'] = worksheet
worksheet.write(0,0,"DVS Selected")
dvs_df.to_excel(writer,sheet_name='Compiled',startrow=2 , startcol=0)

workbook.close()